In [1]:
import os, sys, pickle

import pandas as pd
import numpy as np

from datetime import datetime as date

from sklearn.model_selection import KFold, train_test_split, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import log_loss, roc_auc_score, auc, roc_curve

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%config IPCompleter.greedy=True

# Read data

In [2]:
df_off = pd.read_csv('data/ccf_offline_stage1_train.csv', keep_default_na=False)
df_off.head(5)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
0,1439408,2632,null,null,0,null,20160217
1,1439408,4663,11002,150:20,1,20160528,null
2,1439408,2632,8591,20:1,0,20160217,null
3,1439408,2632,1078,20:1,0,20160319,null
4,1439408,2632,8591,20:1,0,20160613,null


In [3]:
df_on = pd.read_csv('data/ccf_online_stage1_train.csv', keep_default_na=False)
df_on.head(5)

,User_id,Merchant_id,Action,Coupon_id,Discount_rate,Date_received,Date
0,13740231,18907,2,100017492,500:50,20160513,null
1,13740231,34805,1,null,null,null,20160321
2,14336199,18907,0,null,null,null,20160618
3,14336199,18907,0,null,null,null,20160618
4,14336199,18907,0,null,null,null,20160618


In [4]:
df_test = pd.read_csv('data/ccf_offline_stage1_test_revised.csv', keep_default_na=False)
df_test.head(5)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received
0,4129537,450,9983,30:5,1,20160712
1,6949378,1300,3429,30:5,null,20160706
2,2166529,7113,6928,200:20,5,20160727
3,2166529,7113,1808,100:10,5,20160727
4,6172162,7605,6500,30:1,2,20160708


# Count

In [5]:
print("buy and have coupon:" + str(df_off[(df_off['Date']!='null')&(df_off['Coupon_id']!='null')].shape[0]))
print("buy and don't have coupon:" + str(df_off[(df_off['Date']!='null')&(df_off['Coupon_id']=='null')].shape[0]))
print("don't buy and have coupon:" + str(df_off[(df_off['Date']=='null')&(df_off['Coupon_id']!='null')].shape[0]))
print("don't buy and don't have coupon:" + str(df_off[(df_off['Date']=='null')&(df_off['Coupon_id']=='null')].shape[0]))

buy and have coupon:75382
buy and don't have coupon:701602
don't buy and have coupon:977900
don't buy and don't have coupon:0


# Discount

In [6]:
print("Type of Discount_rate, off: ", df_off['Discount_rate'].unique())
print("Type of Discount_rate, on: ", df_on['Discount_rate'].unique())
print("Type of Discount_rate, test: ", df_test['Discount_rate'].unique())

Type of Discount_rate, off:  ['null' '150:20' '20:1' '200:20' '30:5' '50:10' '10:5' '100:10' '200:30'
 '20:5' '30:10' '50:5' '150:10' '100:30' '200:50' '100:50' '300:30'
 '50:20' '0.9' '10:1' '30:1' '0.95' '100:5' '5:1' '100:20' '0.8' '50:1'
 '200:10' '300:20' '100:1' '150:30' '300:50' '20:10' '0.85' '0.6' '150:50'
 '0.75' '0.5' '200:5' '0.7' '30:20' '300:10' '0.2' '50:30' '200:100'
 '150:5']
Type of Discount_rate, on:  ['500:50' 'null' '150:50' '50:5' '30:1' '300:50' '300:30' '800:50'
 '1000:100' '10:5' '200:30' '20:10' '200:20' '150:10' '300:10' '150:1'
 '200:10' 'fixed' '5:1' '50:10' '100:10' '100:5' '150:20' '200:50' '300:5'
 '500:10' '300:100' '50:30' '50:1' '100:1' '30:20' '10:1' '30:5' '100:50'
 '150:5' '150:30' '300:20' '500:30' '800:100' '50:20' '500:100' '500:20'
 '1000:20' '1000:50' '100:30' '30:10' '200:5' '200:100' '150:100' '200:1'
 '20:5' '1000:10' '20:1' '100:20' '300:1' '500:5' '800:20' '800:30'
 '1000:300' '1000:30' '800:10' '1000:500' '500:300' '1000:5' '800:500']
Ty

打折率分为 3 种情况：

- 'null' 表示没有打折

- [0,1] 表示折扣率

- x:y 表示满x减y

**处理方式：**

- 打折类型：getDiscountType()

- 折扣率：convertRate()

- 满多少：getDiscountMan()

- 减多少：getDiscountJian()

In [7]:
def getDiscountType(row):
    if(row=='null'):
        
        return 'null'

    elif(':' in row):
        
        return 'subtract'
    
    else:
        
        return 'rate'
    
def convertRate(row):
    if(row=='null'):
        
        return 1.0
    
    elif(':' in row):
        
        row = row.split(':')
        
        return 1.0-float(row[1])/float(row[0])
    
    elif(row=='fixed'):
        
        return 'fixed'
    
    else:
        
        return float(row)
        
            
    
def getDiscountReach(row):
    if(':' in row):
        
        row = row.split(':')
        
        return float(row[0])
    
    else:
        
        return 0
    
def getDiscountSubtract(row):
    if(':' in row):
        
        row = row.split(':')
        
        return float(row[1])
    
    else:
        
        return 0

def processingData(df):
    df['discount_type'] = df['Discount_rate'].apply(getDiscountType)
    df['discount_rate'] = df['Discount_rate'].apply(convertRate)
    df['discount_reach'] = df['Discount_rate'].apply(getDiscountReach)
    df['discount_subtract'] = df['Discount_rate'].apply(getDiscountSubtract)
    
    #print(df['discount_rate'].unique())
    
    return df

In [8]:
df_off = processingData(df_off)
df_on = processingData(df_on)
df_test = processingData(df_test)

df_off.head(5)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,discount_type,discount_rate,discount_reach,discount_subtract
0,1439408,2632,null,null,0,null,20160217,null,1.000000,0.0,0.0
1,1439408,4663,11002,150:20,1,20160528,null,subtract,0.866667,150.0,20.0
2,1439408,2632,8591,20:1,0,20160217,null,subtract,0.950000,20.0,1.0
3,1439408,2632,1078,20:1,0,20160319,null,subtract,0.950000,20.0,1.0
4,1439408,2632,8591,20:1,0,20160613,null,subtract,0.950000,20.0,1.0


In [9]:
df_on.head(5)

,User_id,Merchant_id,Action,Coupon_id,Discount_rate,Date_received,Date,discount_type,discount_rate,discount_reach,discount_subtract
0,13740231,18907,2,100017492,500:50,20160513,null,subtract,0.9,500.0,50.0
1,13740231,34805,1,null,null,null,20160321,null,1,0.0,0.0
2,14336199,18907,0,null,null,null,20160618,null,1,0.0,0.0
3,14336199,18907,0,null,null,null,20160618,null,1,0.0,0.0
4,14336199,18907,0,null,null,null,20160618,null,1,0.0,0.0


In [10]:
df_test.head(5)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,discount_type,discount_rate,discount_reach,discount_subtract
0,4129537,450,9983,30:5,1,20160712,subtract,0.833333,30.0,5.0
1,6949378,1300,3429,30:5,null,20160706,subtract,0.833333,30.0,5.0
2,2166529,7113,6928,200:20,5,20160727,subtract,0.900000,200.0,20.0
3,2166529,7113,1808,100:10,5,20160727,subtract,0.900000,100.0,10.0
4,6172162,7605,6500,30:1,2,20160708,subtract,0.966667,30.0,1.0


# Distance

In [11]:
print('Distance type, off：',df_off['Distance'].unique())
print('Distance type, test：',df_test['Distance'].unique())

Distance type, off： ['0' '1' 'null' '2' '10' '4' '7' '9' '3' '5' '6' '8']
Distance type, test： ['1' 'null' '5' '2' '0' '10' '3' '6' '7' '4' '9' '8']


In [12]:
# convert distance
df_off['distance'] = df_off['Distance'].replace('null', -1).astype(int)
print(df_off['distance'].unique())
df_test['distance'] = df_test['Distance'].replace('null', -1).astype(int)
print(df_test['distance'].unique())

[ 0  1 -1  2 10  4  7  9  3  5  6  8]
[ 1 -1  5  2  0 10  3  6  7  4  9  8]


# Date_received

In [13]:
date_received = df_off['Date_received'].unique()
date_received = sorted(date_received[date_received != 'null'])
print("date_received from " + date_received[0] + " to " + date_received[len(date_received)-1])

date_used = df_off['Date'].unique()
date_used = sorted(date_used[date_used != 'null'])
print("date_used from " + date_used[0] + " to " + date_used[len(date_used)-1])

date_received from 20160101 to 20160615
date_used from 20160101 to 20160630


- weekday : {null, 1, 2, 3, 4, 5, 6, 7}

- weekday_type : {1, 0}（周六和周日为1，其他为0）

- Weekday_1 : {1, 0, 0, 0, 0, 0, 0}

- Weekday_2 : {0, 1, 0, 0, 0, 0, 0}

- Weekday_3 : {0, 0, 1, 0, 0, 0, 0}

- Weekday_4 : {0, 0, 0, 1, 0, 0, 0}

- Weekday_5 : {0, 0, 0, 0, 1, 0, 0}

- Weekday_6 : {0, 0, 0, 0, 0, 1, 0}

- Weekday_7 : {0, 0, 0, 0, 0, 0, 1}

In [14]:
def getWeekday(row):
    if(row=='null'):
        
        return row
    
    else:
        
        weekday = date(int(row[0:4]), int(row[4:6]), int(row[6:8])).weekday()
        
        if(weekday==0):
            
            return 7
        
        else:
            
            return weekday
        
def WeekdayTYpe(row):
    if(row=='null'):
        
        return 0
    
    else:
        
        return 1

def OneHotWeekday(df):
    onehot = pd.get_dummies(df['weekday'].replace('null', np.nan))
    weekdaycols = ['weekday_' + str(i) for i in range(1,8)]
    
    onehot.columns = [weekdaycols]
    df[weekdaycols] = onehot
    
    return df

In [17]:
df_off['weekday'] = df_off['Date_received'].astype(str).apply(getWeekday)
df_test['weekday'] = df_test['Date_received'].astype(str).apply(getWeekday)

df_off['weekday_type'] = df_off['weekday'].astype(str).apply(WeekdayTYpe)
df_test['weekday_type'] = df_test['weekday'].astype(str).apply(WeekdayTYpe)

df_off = OneHotWeekday(df_off)
df_test = OneHotWeekday(df_test)

df_off.head(5)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,discount_type,discount_rate,discount_reach,...,distance,weekday,weekday_type,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7
0,1439408,2632,null,null,0,null,20160217,null,1.000000,0.0,...,0,null,0,0,0,0,0,0,0,0
1,1439408,4663,11002,150:20,1,20160528,null,subtract,0.866667,150.0,...,1,5,1,0,0,0,0,1,0,0
2,1439408,2632,8591,20:1,0,20160217,null,subtract,0.950000,20.0,...,0,2,1,0,1,0,0,0,0,0
3,1439408,2632,1078,20:1,0,20160319,null,subtract,0.950000,20.0,...,0,5,1,0,0,0,0,1,0,0
4,1439408,2632,8591,20:1,0,20160613,null,subtract,0.950000,20.0,...,0,7,1,0,0,0,0,0,0,1


# features

- discount_rate

- discount_type

- discount_reach

- discount_substract

- distance

- weekday

- weekday_type

- weekday_1

- weekday_2

- weekday_3

- weekday_4

- weekday_5

- weekday_6

- weekday_7

# labels

3 cases：

- Date_received == 'null'：y = -1

- (Date_received != 'null') & (Date != 'null') & (Date - Date_received <= 15)：use coupon in 15 days，positive samples，y = 1

- (Date_received != 'null') & ((Date == 'null') | (Date - Date_received > 15))：don't use coupon in 15 days，negative samples，y = 0

In [20]:
def label(row):
    if(row['Date_received']=='null'):
        
        return -1
    
    if row['Date'] != 'null':
        
        td = pd.to_datetime(row['Date'], format='%Y%m%d') - pd.to_datetime(row['Date_received'], format='%Y%m%d')
        
        if td <= pd.Timedelta(15, 'D'):
            
            return 1
    
    return 0

df_off['label'] = df_off.apply(label, axis=1)

df_off.head(5)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,discount_type,discount_rate,discount_reach,...,weekday,weekday_type,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7,label
0,1439408,2632,null,null,0,null,20160217,null,1.000000,0.0,...,null,0,0,0,0,0,0,0,0,-1
1,1439408,4663,11002,150:20,1,20160528,null,subtract,0.866667,150.0,...,5,1,0,0,0,0,1,0,0,0
2,1439408,2632,8591,20:1,0,20160217,null,subtract,0.950000,20.0,...,2,1,0,1,0,0,0,0,0,0
3,1439408,2632,1078,20:1,0,20160319,null,subtract,0.950000,20.0,...,5,1,0,0,0,0,1,0,0,0
4,1439408,2632,8591,20:1,0,20160613,null,subtract,0.950000,20.0,...,7,1,0,0,0,0,0,0,1,0


# dataset

- training set：20160101-20160515；validation set：20160516-20160615。

In [24]:
# data split
df = df_off[df_off['label'] != -1].copy()
df = df.sample(frac=1)
training = df[:int(df.shape[0]*0.75)].copy()
validation = df[int(df.shape[0]*0.75):].copy()
print('Training Set: \n', training['label'].value_counts())
print('Validation Set: \n', validation['label'].value_counts())

Training Set: 
 0    741540
1     48421
Name: label, dtype: int64
Validation Set: 
 0    247347
1     15974
Name: label, dtype: int64


In [26]:
# feature
weekdaycols = ['weekday_' + str(i) for i in range(1,8)]
original_feature = ['discount_rate','discount_type','discount_man', 'discount_jian','distance', 'weekday', 'weekday_type'] + weekdaycols
print(original_feature)

['discount_rate', 'discount_type', 'discount_man', 'discount_jian', 'distance', 'weekday', 'weekday_type', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'weekday_7']


# model

In [27]:
def check_model(data, predictors):
    
    classifier = lambda: SGDClassifier(
        loss='log',  # loss function: logistic regression
        penalty='elasticnet', # L1 & L2
        fit_intercept=True,  # 是否存在截距，默认存在
        max_iter=100, 
        shuffle=True,  # Whether or not the training data should be shuffled after each epoch
        n_jobs=1, # The number of processors to use
        class_weight=None) # Weights associated with classes. If not given, all classes are supposed to have weight one.
 
    # 管道机制使得参数集在新数据集（比如测试集）上的重复使用，管道机制实现了对全部步骤的流式化封装和管理。
    model = Pipeline(steps=[
        ('ss', StandardScaler()), # transformer
        ('en', classifier())  # estimator
    ])
 
    parameters = {
        'en__alpha': [ 0.001, 0.01, 0.1],
        'en__l1_ratio': [ 0.001, 0.01, 0.1]
    }
 
    # StratifiedKFold用法类似Kfold，但是他是分层采样，确保训练集，测试集中各类别样本的比例与原始数据集中相同。
    folder = StratifiedKFold(n_splits=3, shuffle=True)
    
    # Exhaustive search over specified parameter values for an estimator.
    grid_search = GridSearchCV(
        model, 
        parameters, 
        cv=folder, 
        n_jobs=-1,  # -1 means using all processors
        verbose=1)
    grid_search = grid_search.fit(data[predictors], 
                                  data['label'])
    
    return grid_search